In [ ]:
# Essential imports

import numpy as np
import pandas as pd 
import re


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from wordcloud import WordCloud, STOPWORDS 
stopwords = set(STOPWORDS) 
import matplotlib.pyplot as plt

In [ ]:
dataFrame = pd.read_csv('/kaggle/input/flipkart-customer-review-and-rating/data.csv')
##showing first 5 reviews
dataFrame[0:5]

In [ ]:
## plotting bar graph for 
plt.figure(figsize = (7,7))
ratings = ['1star','2star','3star', '4star', '5star']
numRatings = [len(dataFrame[dataFrame['rating'] == 1 ]),
             len(dataFrame[dataFrame['rating'] == 2 ]),
             len(dataFrame[dataFrame['rating'] == 3 ]),
             len(dataFrame[dataFrame['rating'] == 4 ]),
             len(dataFrame[dataFrame['rating'] == 5 ])
             ]
plt.title('Biased Dataset Alert!!')
plt.grid(True)
plt.bar(ratings, numRatings)
plt.show()

In [ ]:
# to tackle first task lets define a useFul 
# Function
def WordCloudForRating (dataFrame, rating):
    df =  (dataFrame[dataFrame['rating'] == rating]['review'])
    corpus = ''
    
    for mem in df:
        corpus += mem
    
    corpus = re.sub('READ MORE', '', corpus)
    wordcloud = WordCloud(width = 800, height = 800,background_color ='grey',
                          stopwords = stopwords,  min_font_size = 10).generate(corpus)
    plt.figure(figsize = (8, 8), facecolor = None) 
    plt.imshow(wordcloud) 
    plt.rcParams.update({'font.size': 25})
    plt.axis("off") 
    plt.title('Word Cloud:  ' + str(rating) + ' star reviews')
    plt.tight_layout(pad = 0) 
  
    plt.show() 
    

In [ ]:
# word cloud for 1 star products
WordCloudForRating(dataFrame, 1)

In [ ]:
# word cloud for 2 star products
WordCloudForRating(dataFrame, 2)

In [ ]:
# word cloud for 3 star products
WordCloudForRating(dataFrame, 3)

In [ ]:
# word cloud for 4 star products
WordCloudForRating(dataFrame, 4)

In [ ]:
# word cloud for 5 star products
WordCloudForRating(dataFrame, 5)

In [ ]:
## lets predict Product ratings 
## will be using mutliclass classification for 
## for our Purposue

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

text = dataFrame['review']
label = dataFrame['rating']

## Converting data to numpy array
## for ease of analysis

text = np.array(text)
label = np.array(label)
## also labels are from 1-5
## tf expects 0-4
## hence this
label = label - 1

## splitting to train and test sets
X_train, X_test, y_train, y_test = train_test_split(text, label, test_size = 0.1, random_state = 42)

tokenizer = Tokenizer(num_words = 25000)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_train = pad_sequences(X_train, maxlen = 50)
X_test = pad_sequences(X_test, maxlen = 50)
len(tokenizer.word_index)


In [ ]:
## defining model 
Mymodel = tf.keras.models.Sequential([
    
    tf.keras.layers.Input(shape = (50,)),
    tf.keras.layers.Embedding(7857, 20),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(70, return_sequences = True)),
    tf.keras.layers.LSTM(140),
    tf.keras.layers.Dense(5, activation = 'softmax')
])

Mymodel.compile(optimizer = 'Adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'] )

In [ ]:
## training model 
retVal = Mymodel.fit(X_train, y_train, validation_data = (X_test,y_test), epochs = 20 )

In [ ]:
plt.figure(figsize = (7,7))
plt.plot(retVal.history['loss'], label = 'training_loss')
plt.plot(retVal.history['val_loss'], label = 'validation_loss')
plt.legend()
plt.grid(True)

In [ ]:
plt.figure(figsize = (7,7))
plt.plot(retVal.history['accuracy'], label = 'training_accuracy')
plt.plot(retVal.history['val_accuracy'], label = 'val_accuracy')
plt.legend()
plt.grid(True)